In [1]:
!pip install transformers

In [12]:
import numpy as np
import torch
import pandas as pd
import transformers
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split,cross_val_score, GridSearchCV
from sklearn.metrics import accuracy_score

In [3]:
df = pd.read_csv('https://github.com/clairett/pytorch-sentiment-classification/raw/master/data/SST2/train.tsv', delimiter='\t', header=None)
df

,0,1
0,"a stirring , funny and finally transporting re...",1
1,apparently reassembled from the cutting room f...,0
2,they presume their audience wo n't sit still f...,0
3,this is a visually stunning rumination on love...,1
4,jonathan parker 's bartleby should have been t...,1
...,...,...
6915,"painful , horrifying and oppressively tragic ,...",1
6916,take care is nicely performed by a quintet of ...,0
6917,"the script covers huge , heavy topics in a bla...",0
6918,a seriously bad film with seriously warped log...,0


In [4]:
df = df[:5000]

In [5]:
from transformers import DistilBertModel,DistilBertTokenizer
pretrained_weights = 'distilbert-base-uncased'

model = DistilBertModel.from_pretrained(pretrained_weights)
tokenizer = DistilBertTokenizer.from_pretrained(pretrained_weights)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
d = tokenizer(list(df[0]),truncation=True,padding=True)
X = np.array(d['input_ids'])
print(X.shape)
attention = np.array(d['attention_mask'])
print(attention.shape)

(5000, 67)
(5000, 67)


In [7]:
print(attention[0])
print(X[0])

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
[  101  1037 18385  1010  6057  1998  2633 18276  2128 16603  1997  5053
  1998  1996  6841  1998  5687  5469  3152   102     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0     0     0     0     0     0
     0     0     0     0     0     0     0]


In [8]:
input_ids = torch.tensor(X)
attention_mask = torch.tensor(attention)

with torch.no_grad():
  last_hidden_states = model(input_ids,attention_mask=attention_mask)

In [9]:
features = last_hidden_states[0][:,0,:].numpy()
labels = df[1]

In [10]:
X_train, X_test, y_train, y_test = train_test_split(features,labels,test_size=0.2,random_state=1)

In [19]:
# params = {'C':np.linspace(0.0001,100,20)}
# grid_search = GridSearchCV(LogisticRegression(),param_grid=params)
# grid_search.fit(X_train,y_train)
# print(grid_search.best_params_)
# print(grid_search.best_score_)

In [20]:
lr = LogisticRegression(C=10.526405263157894,max_iter=10000)
lr.fit(X_train,y_train)
print(accuracy_score(lr.predict(X_train),y_train))
print(accuracy_score(lr.predict(X_test),y_test))

0.90925
0.834
